In [7]:
import pandas as pd
import pymysql

# DB 연결
conn = pymysql.connect(
    host="localhost", # MariaDB 서버 주소
    port=3306,
    user="webuser",       # 사용자명
    password="webuser",  # 비밀번호
    database="DDProject",  # 사용할 데이터베이스
    charset="utf8mb4"  # 한글 지원
)



# transactional
try:
    conn.autocommit(False)
    cursor = conn.cursor()
    # 테이블 크리에이트
    create_table_sql = """CREATE TABLE IF NOT EXISTS marketingarea (
        id INT AUTO_INCREMENT PRIMARY KEY,
        상가업소번호 VARCHAR(100),
        상호명 VARCHAR(100),
        상권업종대분류코드 VARCHAR(20),
        상권업종대분류명 VARCHAR(20),
        상권업종중분류코드 VARCHAR(20),
        상권업종중분류명 VARCHAR(20),
        상권업종소분류코드 VARCHAR(20),
        상권업종소분류명 VARCHAR(50),
        시도코드 INT,
        시도명 VARCHAR(20),
        시군구코드 INT,
        시군구명 VARCHAR(20),
        행정동코드 INT,
        행정동명 VARCHAR(20),
        법정동코드 INT,
        법정동명 VARCHAR(20),
        지번코드 VARCHAR(100),
        대지구분코드 INT,
        대지구분명 VARCHAR(20),
        지번본번지 INT,
        지번부번지 FLOAT,
        지번주소 VARCHAR(255),
        도로명코드 VARCHAR(100),
        도로명 VARCHAR(255),
        건물본번지 FLOAT,
        건물부번지 FLOAT,
        건물관리번호 VARCHAR(100),
        건물명 VARCHAR(100),
        도로명주소 VARCHAR(255),
        구우편번호 INT,
        신우편번호 INT,
        층정보 VARCHAR(10),
        경도 FLOAT,
        위도 FLOAT
    )"""

    cursor.execute(create_table_sql)



    # 1. 데이터 로드
    df_area = pd.read_csv('../result/23-12_소상공인시장진흥공단_상가(상권)정보_서울.csv', encoding='utf-8', header=0)
    # 2. 데이터 변환
    df_area = df_area.where(pd.notnull(df_area), 0)
    # 3. 날짜 변환 (문자열 → DATETIME)

    # 테이블 인서트
    placeholders = ', '.join(['%s'] * 34)
    insert_sql = f"""
        INSERT INTO marketingarea (상가업소번호,상호명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,시도코드,시도명,시군구코드,시군구명,행정동코드,행정동명,법정동코드,법정동명,지번코드,대지구분코드,대지구분명,지번본번지,지번부번지,지번주소,도로명코드,도로명,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,층정보,경도,위도
    )
        VALUES ({placeholders})
    """

    data = df_area[df_area.columns.values].itertuples(index=False, name=None)
    cursor.executemany(insert_sql, data)
    # 변경 사항 저장
    conn.commit()

    #transactional error
except pymysql.MySQLError as e:
    print(e)
    conn.rollback()

finally:
    # 연결 종료
    cursor.close()
    conn.close()

In [4]:
df_area.head()

,상가업소번호,상호명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,시도코드,시도명,...,건물본번지,건물부번지,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,층정보,경도,위도
0,MA010120220805430767,토끼정,I2,음식,I201,한식,I20101,백반/한정식,11,서울특별시,...,55.0,0.0,1150010500107740012000001,마커스빌딩,서울특별시 강서구 마곡동로 55,157805,7802,2,126.833022,37.560053
1,MA010120220805430763,카페숲,I2,음식,I212,비알코올,I21201,카페,11,서울특별시,...,129.0,0.0,1154510300109290010016890,0,서울특별시 금천구 시흥대로40길 129,153860,8647,1,126.909939,37.446689
2,MA010120220805430969,다이앤,G2,소매,G209,섬유·의복·신발 소매,G20902,여성 의류 소매업,11,서울특별시,...,13.0,0.0,1114016200107750000009546,제일평화시장,서울특별시 중구 마장로 13,100837,4567,1,127.011381,37.568503
3,MA010120220805431023,숯불에닭,I2,음식,I201,한식,I20110,닭/오리고기 구이/찜,11,서울특별시,...,60.0,0.0,1147010300109710004007555,0,서울특별시 양천구 지양로 60,158846,8040,1,126.834524,37.520990
4,MA010120220805431191,맑은에스테틱,S2,수리·개인,S208,욕탕·신체관리,S20802,마사지/안마,11,서울특별시,...,33.0,0.0,1121510500100480053015378,0,서울특별시 광진구 뚝섬로32길 33,143843,5085,1,127.066814,37.534047
